In [2]:
%pip install streamlit 
%pip install torch 
%pip install transformers 
%pip install sentence-transformers 
%pip install faiss-cpu 
%pip install numpy 
%pip install pypdf2 
%pip install fpdf
%pip install groq
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Import necessary libraries and set up the environment

import os
import torch
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import PyPDF2
from fpdf import FPDF
from functools import lru_cache
from groq import Groq
from dotenv import load_dotenv

In [6]:
# Load environment variables from the .env file
load_dotenv()

# Access the API key
api_key = os.getenv('api_key')

# Set your Groq API key
os.environ["GROQ_API_KEY"] = api_key

# Disable parallelism for Hugging Face tokenizers to avoid deadlocks
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [5]:
# Knowledge base class
class KnowledgeBase:
    def __init__(self):
        self.corpus = []
        self.encoder = SentenceTransformer('all-MiniLM-L6-v2')
        self.index = None

    def add_text(self, text):
        sentences = text.split('.')
        self.corpus.extend([sentence.strip() for sentence in sentences if sentence.strip()])
        self.build_index()

    def build_index(self):
        if self.corpus:
            embeddings = self.encoder.encode(self.corpus)
            dimension = embeddings.shape[1]
            self.index = faiss.IndexFlatL2(dimension)
            self.index.add(embeddings.astype('float32'))

    def retrieve(self, query, k=3):
        if not self.index:
            return []
        query_vector = self.encoder.encode([query])
        _, indices = self.index.search(query_vector.astype('float32'), k)
        return [self.corpus[i] for i in indices[0]]

In [9]:
#Define the KnowledgeBase class

class KnowledgeBase:
    def __init__(self):
        self.corpus = []
        self.encoder = SentenceTransformer('all-MiniLM-L6-v2')
        self.index = None

    def add_text(self, text):
        sentences = text.split('.')
        self.corpus.extend([sentence.strip() for sentence in sentences if sentence.strip()])
        self.build_index()

    def build_index(self):
        if self.corpus:
            embeddings = self.encoder.encode(self.corpus)
            dimension = embeddings.shape[1]
            self.index = faiss.IndexFlatL2(dimension)
            self.index.add(embeddings.astype('float32'))

    def retrieve(self, query, k=3):
        if not self.index:
            return []
        query_vector = self.encoder.encode([query])
        _, indices = self.index.search(query_vector.astype('float32'), k)
        return [self.corpus[i] for i in indices[0]]

In [22]:
# Define the GroqGenerator class

class GroqGenerator:
    def __init__(self, model_name='mixtral-8x7b-32768'):
        self.model_name = model_name
        self.client = Groq()

    def generate_lesson(self, topic, retrieved_content):
        prompt = f"Create an engaging English lesson about {topic}. Use the following information:\n"
        prompt += "\n".join(retrieved_content)
        prompt += "\n\nLesson:"

        chat_completion = self.client.chat.completions.create(
            model=self.model_name,
            messages=[
                {"role": "system",
                 "content": """
                You are an AI English teacher designed to create an elaborative and engaging lesson for students in an online setting. Your role is to generate a comprehensive lesson that helps the student fully understand the chosen topic. Since the lesson will only be created once, it must be detailed, covering all key aspects of the topic. For every topic provided, ensure the lesson includes:

                    Thorough Structure: Organize the lesson with clear sections, including an introduction, key concepts, examples, and a conclusion.
                    In-Depth Explanations: Provide detailed explanations of the topic, breaking down complex concepts into simple, easy-to-understand language, and offering relevant examples to clarify your points.
                    Self-Contained Learning: Ensure that the lesson is elaborative enough to allow the student to fully grasp the topic without needing further assistance.
                    Engaging and Interactive Elements: Although this is a single lesson, include creative learning activities such as vocabulary exercises, comprehension questions, and reflection prompts to enhance understanding.
                    Conversational Tone: Maintain a friendly, supportive, and encouraging tone, as if speaking directly to the student in a one-on-one setting.
                    Real-World Examples: Use relevant, real-world examples and scenarios to make the lesson more relatable and memorable.
                    Encouraging Practice: Conclude the lesson by encouraging the student to review the material and practice further on their own."""},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500,
            temperature=0.7
        )

        return chat_completion.choices[0].message.content

In [23]:
#Define the RAGEnglishTeacher class

class RAGEnglishTeacher:
    def __init__(self, knowledge_base, generator):
        self.knowledge_base = knowledge_base
        self.generator = generator

    @lru_cache(maxsize=32)
    def teach(self, topic):
        relevant_content = self.knowledge_base.retrieve(topic)
        lesson = self.generator.generate_lesson(topic, relevant_content)
        return lesson

In [24]:
def extract_text_from_pdf(pdf_file_path):
    try:
        with open(pdf_file_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()
        return text
    except Exception as e:
        print(f"Error processing PDF: {str(e)}")
        return None

def save_lesson_as_pdf(topic, lesson, output_dir='lessons'):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt=f"Lesson on {topic}", ln=1, align="C")
    pdf.multi_cell(0, 10, txt=lesson)
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    filename = f"{output_dir}/lesson_{topic.replace(' ', '_')}.pdf"
    pdf.output(filename)
    return filename

In [25]:
knowledge_base = KnowledgeBase()
generator = GroqGenerator()
teacher = RAGEnglishTeacher(knowledge_base, generator)

In [26]:

def add_content_to_knowledge_base(content):
    knowledge_base.add_text(content)
    print("Content added to the knowledge base.")


def generate_lesson(topic):
    if not knowledge_base.corpus:
        print("Please add some content to the knowledge base first.")
        return None
    
    lesson = teacher.teach(topic)
    print(f"Lesson on {topic}:\n")
    print(lesson)
    return lesson

In [27]:
def save_lesson(topic, lesson):
    if lesson:
        filename = save_lesson_as_pdf(topic, lesson)
        print(f"Lesson saved as PDF: {filename}")
    else:
        print("No lesson to save.")

In [29]:
pdf_text = extract_text_from_pdf('./A-Students-Introduction-to-English-Grammar (1).pdf')
if pdf_text:
    add_content_to_knowledge_base(pdf_text)

unknown widths : 
[0, IndirectObject(6603, 0, 5676930384)]
unknown widths : 
[0, IndirectObject(6812, 0, 5676930384)]
unknown widths : 
[0, IndirectObject(6700, 0, 5676930384)]
unknown widths : 
[0, IndirectObject(7339, 0, 5676930384)]
unknown widths : 
[0, IndirectObject(7191, 0, 5676930384)]
unknown widths : 
[0, IndirectObject(6603, 0, 5676930384)]
unknown widths : 
[0, IndirectObject(9005, 0, 5676930384)]
unknown widths : 
[0, IndirectObject(6643, 0, 5676930384)]
unknown widths : 
[0, IndirectObject(7239, 0, 5676930384)]
unknown widths : 
[0, IndirectObject(6588, 0, 5676930384)]
unknown widths : 
[0, IndirectObject(6633, 0, 5676930384)]
unknown widths : 
[0, IndirectObject(6712, 0, 5676930384)]
unknown widths : 
[0, IndirectObject(6588, 0, 5676930384)]
unknown widths : 
[0, IndirectObject(7191, 0, 5676930384)]
unknown widths : 
[0, IndirectObject(6643, 0, 5676930384)]
unknown widths : 
[0, IndirectObject(6603, 0, 5676930384)]
unknown widths : 
[0, IndirectObject(6613, 0, 5676930384

Content added to the knowledge base.


In [32]:
lesson = generate_lesson("Verbs")

Lesson on Verbs:

Introduction
Hello there! Today, we're going to dive into an exciting topic: Verbs! Verbs are action words that help us express what's happening in a sentence. They can describe actions, states of being, and occurrences. By the end of this lesson, you'll have a solid understanding of verbs and how they contribute to the English language.

Key Concepts
1. Definition of Verbs: Verbs are words that indicate action, state of being, or occurrence. They can be categorized into transitive and intransitive verbs based on whether they take an object or not.

2. Types of Verbs: There are many types of verbs in English, including:
   a. Action Verbs: These describe physical or mental actions (e.g., run, think, write).
   b. Linking Verbs: These connect the subject to additional information (e.g., am, is, are, was, were, be, being, been).
   c. Helping Verbs: These assist linking verbs to create verb tenses, voice, and mood (e.g., has, have, had, do, does, did).
   d. Stative Ver

In [33]:
save_lesson("Verbs", lesson)

Lesson saved as PDF: lessons/lesson_Verbs.pdf
